In [ ]:
import argparse
import os
import sys
import pickle
import time
import subprocess
import shutil
sys.path.append(os.getcwd())

from khrylib.utils import *
from khrylib.rl.utils.visualizer import Visualizer
from khrylib.rl.core.policy_gaussian import PolicyGaussian
from khrylib.rl.core.critic import Value
from khrylib.models.mlp import MLP
from motion_imitation.envs.humanoid_im import HumanoidEnv
from motion_imitation.utils.config import Config

# parser = argparse.ArgumentParser()
# parser.add_argument('--cfg', default=None)
# parser.add_argument('--vis_model_file', default='mocap_v2_vis')
# parser.add_argument('--iter', type=int, default=-1)
# parser.add_argument('--focus', action='store_true', default=True)
# parser.add_argument('--hide_expert', action='store_true', default=False)
# parser.add_argument('--preview', action='store_true', default=False)
# parser.add_argument('--record', action='store_true', default=False)
# parser.add_argument('--record_expert', action='store_true', default=False)
# parser.add_argument('--azimuth', type=float, default=45)
# parser.add_argument('--video_dir', default='out/videos/motion_im')
args = parser.parse_args()
cfg = Config(args.cfg, False, create_dirs=False)
cfg.env_start_first = True
logger = create_logger(os.path.join(cfg.log_dir, 'log_eval.txt'))

"""make and seed env"""
dtype = torch.float64
torch.set_default_dtype(dtype)
torch.manual_seed(cfg.seed)
torch.set_grad_enabled(False)
env = HumanoidEnv(cfg)
env.seed(cfg.seed)


NameError: name 'fullpath' is not defined